# 🔬 Multi-Model Comparison Analysis for Tourism Mobility Prediction

*Automated analysis and visualization for comparing multiple LLM models on VeronaCard dataset*

---

## 🎯 Obiettivi del Notebook

Questo notebook fornisce un **framework completo** per confrontare le performance di diversi modelli LLM sulla predizione next-POI turistici, generando automaticamente:

- **📊 Grafici comparativi** pronti per inclusione nella tesi
- **📈 Analisi statistiche** dettagliate per ogni modello
- **🎨 Visualizzazioni professionali** con stili consistenti
- **📋 Report automatici** in formato LaTeX-ready

## 📁 Struttura Repository Richiesta

**⚠️ IMPORTANTE**: Prima di eseguire questo notebook, organizza la repository come segue:

```
LLM-Mob-As-Mobility-Interpreter/
├── notebook/
│   ├── veronacard_analysis.ipynb              # tuo notebook esistente
│   └── multi_model_comparison_analysis.ipynb  # questo notebook
├── results/                                   # NUOVA CARTELLA DA CREARE
│   ├── llama3.1_8b/
│   │   ├── dati_2014_pred_*.csv
│   │   ├── dati_2015_pred_*.csv
│   │   └── ... (tutti i file CSV per LLaMA)
│   ├── mixtral_8x7b/
│   │   ├── dati_2014_pred_*.csv
│   │   └── ... (tutti i file CSV per Mixtral)
│   ├── qwen2.5_7b/
│   │   └── ... (file CSV per Qwen)
│   ├── deepseek_v3_8b/
│   │   └── ... (file CSV per DeepSeek)
│   ├── gemma3_8b/
│   │   └── ... (file CSV per Gemma 8B)
│   └── gemma3_27b/
│       └── ... (file CSV per Gemma 27B)
└── img/                                       # cartelle già create
    ├── multi_model_comparison/
    ├── llama3.1_8b_extended/
    └── ... (altre cartelle modelli)
```

## 🚀 Workflow di Utilizzo

1. **Prepara i dati**: Sposta tutti i file `*_pred_*.csv` nelle rispettive cartelle sotto `results/`
2. **Configura il notebook**: Modifica la lista `MODELS` se necessario
3. **Esegui tutte le celle**: Il notebook genererà automaticamente tutti i grafici
4. **Verifica output**: Controlla che i file vengano salvati in `img/`
5. **Integra in LaTeX**: I grafici sono pronti per l'inclusione nella tesi

---

## 📦 Setup e Configurazione

### Configurazione dei modelli da analizzare
Modifica questa configurazione in base ai modelli che hai testato:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import ast
import json
import warnings
from collections import Counter, defaultdict
import os

# Configurazione stile grafici
plt.rcParams['figure.dpi'] = 300  # Alta qualità per la tesi
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 11

# Sopprime warning non critici
warnings.filterwarnings('ignore', category=UserWarning)

print("📦 Setup completato!")
print(f"📊 Working directory: {os.getcwd()}")

### Configurazione modelli e colori

In [ ]:
# ============================================================================
# CONFIGURAZIONE MODELLI - Modifica questa sezione se necessario
# ============================================================================

MODELS = {
    'llama3.1_8b': {
        'name': 'LLaMA 3.1 8B',
        'dir': '../results/llama3.1_8b/',
        'color': '#1f77b4',  # Blu
        'params': '8B',
        'type': 'Baseline'
    },
    'mixtral_8x7b': {
        'name': 'Mixtral 8x7B',
        'dir': '../results/mixtral_8x7b/',
        'color': '#ff7f0e',  # Arancione
        'params': '8x7B (47B active)',
        'type': 'MoE'
    },
    'qwen2.5_7b': {
        'name': 'Qwen 2.5 7B',
        'dir': '../results/qwen2.5_7b/',
        'color': '#2ca02c',  # Verde
        'params': '7B',
        'type': 'Multilingual'
    },
    'deepseek_v3_8b': {
        'name': 'DeepSeek-V3 8B',
        'dir': '../results/deepseek_v3_8b/',
        'color': '#d62728',  # Rosso
        'params': '8B',
        'type': 'Reasoning'
    },
    'gemma3_8b': {
        'name': 'Gemma 3 8B',
        'dir': '../results/gemma3_8b/',
        'color': '#9467bd',  # Viola
        'params': '8B',
        'type': 'Google'
    },
    'gemma3_27b': {
        'name': 'Gemma 3 27B',
        'dir': '../results/gemma3_27b/',
        'color': '#8c564b',  # Marrone
        'params': '27B',
        'type': 'Large-scale'
    }
}

# Directory per salvare i grafici
OUTPUT_DIRS = {
    'comparison': '../img/multi_model_comparison/',
    'individual': '../img/'  # + model_key per grafici specifici
}

# Crea directory se non esistono
for dir_path in OUTPUT_DIRS.values():
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print("🎨 Configurazione modelli completata:")
for key, model in MODELS.items():
    print(f"  {model['name']:>15s} → {model['dir']} [{model['color']}]")
    
print(f"\n📁 Output directory: {OUTPUT_DIRS['comparison']}")

## 📈 1. Caricamento e Preprocessing Dati

### Funzioni di utilità per il caricamento dati

In [ ]:
def poi_id(x):
    """
    Normalizza identificatori POI per confronti consistenti.
    Gestisce dict, liste, e stringhe/numeri.
    """
    if isinstance(x, dict):
        for key in ('poi', 'poi_id', 'name', 'id'):
            if key in x:
                return str(x[key])
        return json.dumps(x, sort_keys=True)
    elif isinstance(x, (list, tuple)):
        return tuple(map(poi_id, x))
    else:
        return str(x)

def load_model_data(model_dir, model_name):
    """
    Carica tutti i CSV di un modello e calcola le metriche.
    
    Args:
        model_dir (str): Directory contenente i file CSV del modello
        model_name (str): Nome del modello per logging
    
    Returns:
        pd.DataFrame: DataFrame con tutte le predizioni e metriche calcolate
    """
    print(f"📂 Caricamento {model_name}...")
    
    # Trova tutti i file CSV
    csv_pattern = str(Path(model_dir) / '*_pred_*.csv')
    csv_files = glob.glob(csv_pattern)
    
    if not csv_files:
        print(f"   ⚠️  Nessun file CSV trovato in {model_dir}")
        print(f"       Pattern cercato: {csv_pattern}")
        return pd.DataFrame()
    
    print(f"   📄 Trovati {len(csv_files)} file CSV")
    
    dfs = []
    total_rows = 0
    
    for csv_file in sorted(csv_files):
        try:
            # Caricamento con gestione errori
            try:
                df = pd.read_csv(csv_file)
            except pd.errors.ParserError:
                print(f"   ⚠️  Parser error in {Path(csv_file).name}, utilizzando error handling...")
                df = pd.read_csv(csv_file, on_bad_lines='skip', engine='python')
            
            # Estrai anno dal nome file
            filename = Path(csv_file).stem
            year_token = next((part for part in filename.split('_')
                             if part.isdigit() and len(part) == 4), None)
            df['year'] = int(year_token) if year_token else np.nan
            df['model'] = model_name
            
            # Parse prediction list
            df['prediction_list'] = df['prediction'].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else []
            )
            
            # Filtra solo predizioni con esattamente 5 elementi
            df = df[df['prediction_list'].apply(len) == 5]
            
            dfs.append(df)
            total_rows += len(df)
            print(f"       {Path(csv_file).name}: {len(df):,} righe")
            
        except Exception as e:
            print(f"   ❌ Errore caricando {Path(csv_file).name}: {str(e)}")
            continue
    
    if not dfs:
        print(f"   ❌ Nessun file caricato con successo per {model_name}")
        return pd.DataFrame()
    
    # Concatena tutti i dataframes
    df_combined = pd.concat(dfs, ignore_index=True)
    
    # Normalizza POI per confronti consistenti
    df_combined['prediction_norm'] = df_combined['prediction_list'].apply(
        lambda lst: [poi_id(e) for e in lst]
    )
    df_combined['ground_truth_norm'] = df_combined['ground_truth'].apply(poi_id)
    
    print(f"   ✅ {model_name}: {total_rows:,} righe totali")
    return df_combined

def calculate_metrics(df):
    """
    Calcola le metriche di valutazione standard per un DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame con colonne prediction_norm e ground_truth_norm
    
    Returns:
        dict: Dizionario con le metriche calcolate
    """
    if df.empty:
        return {
            'top1_accuracy': 0.0,
            'top5_hit_rate': 0.0,
            'mrr': 0.0,
            'catalogue_coverage': 0.0,
            'total_predictions': 0,
            'processing_time_mean': 0.0
        }
    
    # Calcola hit@1
    df['hit@1'] = df['prediction_norm'].str[0] == df['ground_truth_norm']
    
    # Calcola hit@5
    df['hit@5'] = df.apply(
        lambda row: row['ground_truth_norm'] in row['prediction_norm'][:5], axis=1
    )
    
    # Calcola reciprocal rank
    def reciprocal_rank(row, k=5):
        try:
            rank = row['prediction_norm'][:k].index(row['ground_truth_norm']) + 1
            return 1.0 / rank
        except ValueError:
            return 0.0
    
    df['rr'] = df.apply(reciprocal_rank, axis=1)
    
    # Calcola catalogue coverage
    coverage_set = {poi for preds in df['prediction_norm'] for poi in preds}
    unique_ground_truth = df['ground_truth_norm'].nunique()
    
    # Processing time medio
    proc_time_mean = df['processing_time'].mean() if 'processing_time' in df.columns else 0.0
    
    metrics = {
        'top1_accuracy': df['hit@1'].mean(),
        'top5_hit_rate': df['hit@5'].mean(),
        'mrr': df['rr'].mean(),
        'catalogue_coverage': len(coverage_set) / unique_ground_truth if unique_ground_truth > 0 else 0.0,
        'total_predictions': len(df),
        'processing_time_mean': proc_time_mean
    }
    
    return metrics

print("🔧 Funzioni di utilità caricate!")

### Caricamento dati per tutti i modelli

In [ ]:
# ============================================================================
# CARICAMENTO DATI PER TUTTI I MODELLI
# ============================================================================

print("🚀 Avvio caricamento dati multi-modello...\n")
print("=" * 60)

# Dizionario per contenere tutti i dati
model_data = {}
model_metrics = {}
successfully_loaded = []

# Carica dati per ogni modello
for model_key, model_config in MODELS.items():
    print(f"\n📊 MODELLO: {model_config['name']}")
    print("-" * 40)
    
    # Verifica esistenza directory
    model_dir = Path(model_config['dir'])
    if not model_dir.exists():
        print(f"   ❌ Directory non trovata: {model_dir}")
        print(f"   💡 Crea la directory e inserisci i file CSV del modello")
        continue
    
    # Carica i dati
    df = load_model_data(model_config['dir'], model_config['name'])
    
    if df.empty:
        print(f"   ⏭️  Saltando {model_config['name']} (nessun dato)")
        continue
    
    # Calcola metriche
    metrics = calculate_metrics(df)
    
    # Salva nei dizionari
    model_data[model_key] = df
    model_metrics[model_key] = metrics
    successfully_loaded.append(model_key)
    
    # Report per questo modello
    print(f"   ✅ Metriche {model_config['name']}:")
    print(f"       Top-1 Accuracy: {metrics['top1_accuracy']:.3f} ({metrics['top1_accuracy']*100:.1f}%)")
    print(f"       Top-5 Hit Rate: {metrics['top5_hit_rate']:.3f} ({metrics['top5_hit_rate']*100:.1f}%)")
    print(f"       MRR:            {metrics['mrr']:.3f}")
    print(f"       Coverage:       {metrics['catalogue_coverage']:.2f}")
    print(f"       Predizioni:     {metrics['total_predictions']:,}")
    
    if metrics['processing_time_mean'] > 0:
        print(f"       Tempo/Card:     {metrics['processing_time_mean']:.2f}s")

print("\n" + "=" * 60)
print(f"📋 RIEPILOGO CARICAMENTO:")
print(f"   Modelli configurati: {len(MODELS)}")
print(f"   Modelli caricati:    {len(successfully_loaded)}")
print(f"   Modelli disponibili: {', '.join([MODELS[k]['name'] for k in successfully_loaded])}")

if len(successfully_loaded) < 2:
    print("\n⚠️  ATTENZIONE: Servono almeno 2 modelli per i confronti!")
    print("   Verifica la struttura delle directory in results/")
else:
    print(f"\n✅ Pronto per analisi multi-modello con {len(successfully_loaded)} modelli!")

## 📊 2. Creazione Tabella Comparativa Principale

Genera la tabella di performance comparative che sarà integrata nel LaTeX.

In [ ]:
# ============================================================================
# TABELLA COMPARATIVA PRINCIPALE
# ============================================================================

if len(successfully_loaded) == 0:
    print("❌ Nessun modello caricato. Saltando creazione tabella.")
else:
    print("📋 Creazione tabella comparativa...")
    
    # Prepara dati per la tabella
    comparison_data = []
    
    for model_key in successfully_loaded:
        model_config = MODELS[model_key]
        metrics = model_metrics[model_key]
        
        # Calcola cards per second se disponibile processing time
        if metrics['processing_time_mean'] > 0:
            cards_per_sec = 1.0 / metrics['processing_time_mean']
            time_display = f"{metrics['processing_time_mean']:.1f}s"
            efficiency = f"{cards_per_sec:.1f}"
        else:
            time_display = "N/A"
            efficiency = "N/A"
        
        comparison_data.append({
            'Modello': model_config['name'],
            'Parametri': model_config['params'],
            'Top-1 Acc.': f"{metrics['top1_accuracy']*100:.1f}%",
            'Top-5 HR': f"{metrics['top5_hit_rate']*100:.1f}%",
            'MRR': f"{metrics['mrr']*100:.1f}%",
            'Coverage': f"{metrics['catalogue_coverage']:.2f}",
            'Tempo/Card': time_display,
            'Cards/Sec': efficiency,
            'Predizioni': f"{metrics['total_predictions']:,}"
        })
    
    # Crea DataFrame per display
    comparison_df = pd.DataFrame(comparison_data)
    
    print("\n📊 TABELLA COMPARATIVA PERFORMANCE:")
    print("=" * 80)
    print(comparison_df.to_string(index=False))
    print("=" * 80)
    
    # Salva la tabella per uso in LaTeX
    latex_table_path = Path(OUTPUT_DIRS['comparison']) / 'performance_table.csv'
    comparison_df.to_csv(latex_table_path, index=False)
    print(f"\n💾 Tabella salvata in: {latex_table_path}")
    
    # Genera anche codice LaTeX pronto all'uso
    latex_code_path = Path(OUTPUT_DIRS['comparison']) / 'performance_table.tex'
    
    with open(latex_code_path, 'w', encoding='utf-8') as f:
        f.write("% Tabella generata automaticamente\n")
        f.write("\\begin{table}[H]\n")
        f.write("\\centering\n")
        f.write("\\caption{Performance Comparative tra Modelli LLM (Strategia Geospaziale)}\n")
        f.write("\\label{tab:models_comparison}\n")
        f.write("\\begin{tabular}{lcccccc}\n")
        f.write("\\toprule\n")
        f.write("\\textbf{Modello} & \\textbf{Parametri} & \\textbf{Top-1 Acc.} & \\textbf{Top-5 HR} & \\textbf{MRR} & \\textbf{Coverage} & \\textbf{Tempo/Card} \\\\\n")
        f.write("\\midrule\n")
        
        for _, row in comparison_df.iterrows():
            f.write(f"{row['Modello']} & {row['Parametri']} & {row['Top-1 Acc.']} & {row['Top-5 HR']} & {row['MRR']} & {row['Coverage']} & {row['Tempo/Card']} \\\\\n")
        
        f.write("\\bottomrule\n")
        f.write("\\end{tabular}\n")
        f.write("\\end{table}\n")
    
    print(f"📄 Codice LaTeX salvato in: {latex_code_path}")
    
    # Identifica il modello best-performing per ogni metrica
    print("\n🏆 MODELLI BEST-PERFORMING PER METRICA:")
    print("-" * 50)
    
    # Top-1 Accuracy
    best_top1 = max(successfully_loaded, key=lambda k: model_metrics[k]['top1_accuracy'])
    print(f"Top-1 Accuracy: {MODELS[best_top1]['name']} ({model_metrics[best_top1]['top1_accuracy']*100:.1f}%)")
    
    # Top-5 Hit Rate
    best_top5 = max(successfully_loaded, key=lambda k: model_metrics[k]['top5_hit_rate'])
    print(f"Top-5 Hit Rate: {MODELS[best_top5]['name']} ({model_metrics[best_top5]['top5_hit_rate']*100:.1f}%)")
    
    # MRR
    best_mrr = max(successfully_loaded, key=lambda k: model_metrics[k]['mrr'])
    print(f"MRR:            {MODELS[best_mrr]['name']} ({model_metrics[best_mrr]['mrr']*100:.1f}%)")
    
    # Efficienza (se disponibile)
    models_with_time = [k for k in successfully_loaded if model_metrics[k]['processing_time_mean'] > 0]
    if models_with_time:
        best_efficiency = min(models_with_time, key=lambda k: model_metrics[k]['processing_time_mean'])
        print(f"Efficienza:     {MODELS[best_efficiency]['name']} ({model_metrics[best_efficiency]['processing_time_mean']:.1f}s/card)")

## 🎨 3. Generazione Grafici Comparativi Principali

### 3.1 Grafico a Barre Comparativo Principale

In [ ]:
# ============================================================================
# GRAFICO COMPARATIVO PRINCIPALE - Barre Raggruppate
# ============================================================================

if len(successfully_loaded) < 2:
    print("⏭️  Saltando grafici: servono almeno 2 modelli")
else:
    print("🎨 Creazione grafico comparativo principale...")
    
    # Prepara i dati per il grafico
    models_list = [MODELS[k]['name'] for k in successfully_loaded]
    colors_list = [MODELS[k]['color'] for k in successfully_loaded]
    
    top1_values = [model_metrics[k]['top1_accuracy'] * 100 for k in successfully_loaded]
    top5_values = [model_metrics[k]['top5_hit_rate'] * 100 for k in successfully_loaded]
    mrr_values = [model_metrics[k]['mrr'] * 100 for k in successfully_loaded]
    
    # Configura il grafico
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    fig.suptitle('Performance Comparison Across LLM Models', fontsize=20, fontweight='bold')
    
    # Top-1 Accuracy
    bars1 = ax1.bar(models_list, top1_values, color=colors_list, alpha=0.8, edgecolor='black', linewidth=0.5)
    ax1.set_title('Top-1 Accuracy (%)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Accuracy (%)', fontsize=12)
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Aggiungi valori sopra le barre
    for bar, value in zip(bars1, top1_values):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{value:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Top-5 Hit Rate
    bars2 = ax2.bar(models_list, top5_values, color=colors_list, alpha=0.8, edgecolor='black', linewidth=0.5)
    ax2.set_title('Top-5 Hit Rate (%)', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Hit Rate (%)', fontsize=12)
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, value in zip(bars2, top5_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                f'{value:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Mean Reciprocal Rank
    bars3 = ax3.bar(models_list, mrr_values, color=colors_list, alpha=0.8, edgecolor='black', linewidth=0.5)
    ax3.set_title('Mean Reciprocal Rank (%)', fontsize=14, fontweight='bold')
    ax3.set_ylabel('MRR (%)', fontsize=12)
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3, axis='y')
    
    for bar, value in zip(bars3, mrr_values):
        ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{value:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    plt.tight_layout()
    
    # Salva il grafico
    output_path = Path(OUTPUT_DIRS['comparison']) / 'models_performance_comparison.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"💾 Salvato: {output_path}")
    
    plt.show()
    
    # ============================================================================
    # GRAFICO PERFORMANCE SINGOLE METRICHE
    # ============================================================================
    
    print("\n🎨 Creazione grafici per singola metrica...")
    
    metrics_info = [
        ('top1_accuracy', 'Top-1 Accuracy', 'Accuracy (%)', 'top1_accuracy_by_model.png'),
        ('top5_hit_rate', 'Top-5 Hit Rate', 'Hit Rate (%)', 'top5_hitrate_by_model.png'),
        ('mrr', 'Mean Reciprocal Rank', 'MRR (%)', 'mrr_by_model.png')
    ]
    
    for metric_key, title, ylabel, filename in metrics_info:
        fig, ax = plt.subplots(figsize=(10, 6))
        
        values = [model_metrics[k][metric_key] * 100 for k in successfully_loaded]
        
        bars = ax.bar(models_list, values, color=colors_list, alpha=0.8, 
                     edgecolor='black', linewidth=0.5)
        
        ax.set_title(f'{title} - Model Comparison', fontsize=16, fontweight='bold')
        ax.set_ylabel(ylabel, fontsize=12)
        ax.tick_params(axis='x', rotation=45)
        ax.grid(True, alpha=0.3, axis='y')
        
        # Aggiungi valori sopra le barre
        for bar, value in zip(bars, values):
            ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.01, 
                   f'{value:.1f}%', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        
        # Salva
        output_path = Path(OUTPUT_DIRS['comparison']) / filename
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"💾 Salvato: {output_path}")
        
        plt.show()
    
    print("✅ Grafici comparativi principali completati!")

### 3.2 Scatter Plot Performance vs Efficienza

In [ ]:
# ============================================================================
# SCATTER PLOT PERFORMANCE VS EFFICIENZA
# ============================================================================

if len(successfully_loaded) < 2:
    print("⏭️  Saltando scatter plot: servono almeno 2 modelli")
else:
    # Controlla se abbiamo dati sui tempi di processing
    models_with_time = [k for k in successfully_loaded 
                       if model_metrics[k]['processing_time_mean'] > 0]
    
    if len(models_with_time) < 2:
        print("⚠️  Scatter plot performance-efficienza saltato: serve processing_time per almeno 2 modelli")
        print("   💡 Assicurati che i CSV contengano la colonna 'processing_time'")
    else:
        print("🎨 Creazione scatter plot performance vs efficienza...")
        
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Prepara i dati
        x_values = []  # Cards per secondo (efficienza)
        y_values = []  # MRR (performance)
        sizes = []     # Numero di parametri (per dimensione punti)
        colors = []
        labels = []
        
        # Mappa parametri a dimensioni numeriche per i punti
        param_to_size = {
            '7B': 100,
            '8B': 120,
            '8x7B (47B active)': 300,
            '27B': 250
        }
        
        for model_key in models_with_time:
            model_config = MODELS[model_key]
            metrics = model_metrics[model_key]
            
            # Calcola cards per secondo
            cards_per_sec = 1.0 / metrics['processing_time_mean']
            mrr_percent = metrics['mrr'] * 100
            
            x_values.append(cards_per_sec)
            y_values.append(mrr_percent)
            colors.append(model_config['color'])
            labels.append(model_config['name'])
            
            # Dimensione punto basata sui parametri
            size = param_to_size.get(model_config['params'], 150)
            sizes.append(size)
        
        # Crea scatter plot
        scatter = ax.scatter(x_values, y_values, s=sizes, c=colors, alpha=0.7, 
                           edgecolors='black', linewidth=1.5)
        
        # Aggiungi etichette ai punti
        for i, (x, y, label) in enumerate(zip(x_values, y_values, labels)):
            ax.annotate(label, (x, y), xytext=(5, 5), textcoords='offset points',
                       fontsize=10, fontweight='bold')
        
        ax.set_xlabel('Processing Efficiency (Cards/Second)', fontsize=14)
        ax.set_ylabel('Performance (MRR %)', fontsize=14)
        ax.set_title('Performance vs Efficiency Trade-off Analysis', fontsize=16, fontweight='bold')
        ax.grid(True, alpha=0.3)
        
        # Aggiungi linee di riferimento (quartili o mediana)
        if len(x_values) >= 3:
            median_x = np.median(x_values)
            median_y = np.median(y_values)
            ax.axvline(median_x, color='gray', linestyle='--', alpha=0.5, label='Median Efficiency')
            ax.axhline(median_y, color='gray', linestyle='--', alpha=0.5, label='Median Performance')
            ax.legend(loc='best')
        
        # Annotazione sulla dimensione dei punti
        ax.text(0.02, 0.98, 'Point size = Model parameters', transform=ax.transAxes, 
               fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', 
               facecolor='white', alpha=0.8))
        
        plt.tight_layout()
        
        # Salva il grafico
        output_path = Path(OUTPUT_DIRS['comparison']) / 'performance_efficiency_scatter.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"💾 Salvato: {output_path}")
        
        plt.show()
        
        # Analisi dei quadranti
        print("\n📊 ANALISI QUADRANTI (Performance vs Efficienza):")
        print("-" * 50)
        
        if len(x_values) >= 2:
            median_x = np.median(x_values)
            median_y = np.median(y_values)
            
            for i, (x, y, label) in enumerate(zip(x_values, y_values, labels)):
                if x > median_x and y > median_y:
                    quadrant = "🟢 HIGH Performance, HIGH Efficiency (Optimal)"
                elif x > median_x and y <= median_y:
                    quadrant = "🟡 LOW Performance, HIGH Efficiency (Fast but inaccurate)"
                elif x <= median_x and y > median_y:
                    quadrant = "🟠 HIGH Performance, LOW Efficiency (Accurate but slow)"
                else:
                    quadrant = "🔴 LOW Performance, LOW Efficiency (Poor)"
                
                print(f"{label:>15s}: {quadrant}")
        
        print("\n✅ Scatter plot performance-efficienza completato!")

### 3.3 Heat Map Performance

In [ ]:
# ============================================================================
# HEAT MAP PERFORMANCE NORMALIZZATA
# ============================================================================

if len(successfully_loaded) < 2:
    print("⏭️  Saltando heat map: servono almeno 2 modelli")
else:
    print("🎨 Creazione heat map performance normalizzata...")
    
    # Prepara dati per heat map
    metrics_for_heatmap = ['top1_accuracy', 'top5_hit_rate', 'mrr', 'catalogue_coverage']
    metric_labels = ['Top-1\nAccuracy', 'Top-5\nHit Rate', 'Mean Reciprocal\nRank', 'Catalogue\nCoverage']
    
    # Crea matrice dati
    heatmap_data = []
    model_labels = []
    
    for model_key in successfully_loaded:
        model_config = MODELS[model_key]
        metrics = model_metrics[model_key]
        
        row_data = []
        for metric in metrics_for_heatmap:
            if metric == 'catalogue_coverage':
                # Coverage può essere > 1, quindi normalizziamo diversamente
                value = min(metrics[metric] * 100, 100)  # Cap a 100%
            else:
                value = metrics[metric] * 100
            row_data.append(value)
        
        heatmap_data.append(row_data)
        model_labels.append(model_config['name'])
    
    # Converte a numpy array per facilità
    heatmap_matrix = np.array(heatmap_data)
    
    # Crea heat map
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Usa colormap personalizzata
    im = ax.imshow(heatmap_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
    
    # Configura assi
    ax.set_xticks(range(len(metric_labels)))
    ax.set_xticklabels(metric_labels, rotation=0, ha='center')
    ax.set_yticks(range(len(model_labels)))
    ax.set_yticklabels(model_labels)
    
    # Aggiungi valori nelle celle
    for i in range(len(model_labels)):
        for j in range(len(metric_labels)):
            value = heatmap_matrix[i, j]
            text_color = 'white' if value < 50 else 'black'
            ax.text(j, i, f'{value:.1f}%', ha='center', va='center', 
                   color=text_color, fontweight='bold', fontsize=10)
    
    ax.set_title('Performance Heat Map - Model Comparison\n(Normalized 0-100%)', 
                fontsize=16, fontweight='bold', pad=20)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.8, aspect=20)
    cbar.set_label('Performance (%)', rotation=270, labelpad=20, fontsize=12)
    
    plt.tight_layout()
    
    # Salva
    output_path = Path(OUTPUT_DIRS['comparison']) / 'models_performance_heatmap.png'
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"💾 Salvato: {output_path}")
    
    plt.show()
    
    # Analisi best/worst per categoria
    print("\n📊 ANALISI HEAT MAP:")
    print("-" * 40)
    
    for j, metric_name in enumerate(['Top-1 Accuracy', 'Top-5 Hit Rate', 'MRR', 'Coverage']):
        values_for_metric = heatmap_matrix[:, j]
        best_idx = np.argmax(values_for_metric)
        worst_idx = np.argmin(values_for_metric)
        
        print(f"{metric_name}:")
        print(f"  🥇 Best:  {model_labels[best_idx]} ({values_for_metric[best_idx]:.1f}%)")
        print(f"  🥉 Worst: {model_labels[worst_idx]} ({values_for_metric[worst_idx]:.1f}%)")
        print()
    
    print("✅ Heat map performance completata!")

## 📈 4. Analisi Temporale Multi-Modello

Confronto delle performance nel tempo per identificare stabilità e trend.

In [ ]:
# ============================================================================
# ANALISI TEMPORALE MULTI-MODELLO
# ============================================================================

if len(successfully_loaded) < 2:
    print("⏭️  Saltando analisi temporale: servono almeno 2 modelli")
else:
    print("📅 Analisi temporale multi-modello...")
    
    # Verifica che abbiamo dati temporali
    temporal_data_available = False
    for model_key in successfully_loaded:
        df = model_data[model_key]
        if 'year' in df.columns and not df['year'].isna().all():
            temporal_data_available = True
            break
    
    if not temporal_data_available:
        print("   ⚠️  Nessun dato temporale disponibile (colonna 'year' mancante o vuota)")
    else:
        print("   ✅ Dati temporali trovati, creazione grafici...")
        
        # Calcola metriche per anno per ogni modello
        temporal_results = {}
        all_years = set()
        
        for model_key in successfully_loaded:
            df = model_data[model_key]
            if 'year' in df.columns:
                # Filtra anni validi
                df_with_year = df[df['year'].notna()]
                
                if len(df_with_year) > 0:
                    # Calcola metriche per anno
                    yearly_metrics = {}
                    for year in df_with_year['year'].unique():
                        year_data = df_with_year[df_with_year['year'] == year]
                        if len(year_data) > 0:
                            metrics = calculate_metrics(year_data)
                            yearly_metrics[int(year)] = metrics
                            all_years.add(int(year))
                    
                    temporal_results[model_key] = yearly_metrics
        
        if len(temporal_results) == 0:
            print("   ⚠️  Nessun risultato temporale calcolabile")
        else:
            # Ordina gli anni
            sorted_years = sorted(all_years)
            print(f"   📊 Anni disponibili: {sorted_years}")
            
            # Crea grafici temporali
            metrics_to_plot = [
                ('top1_accuracy', 'Top-1 Accuracy (%)', 'temporal_top1_comparison.png'),
                ('top5_hit_rate', 'Top-5 Hit Rate (%)', 'temporal_top5_comparison.png'),
                ('mrr', 'Mean Reciprocal Rank (%)', 'temporal_mrr_comparison.png')
            ]
            
            for metric_key, ylabel, filename in metrics_to_plot:
                fig, ax = plt.subplots(figsize=(12, 6))
                
                # Plot linea per ogni modello
                for model_key in successfully_loaded:
                    if model_key in temporal_results:
                        model_config = MODELS[model_key]
                        
                        # Prepara dati per questo modello
                        x_data = []
                        y_data = []
                        
                        for year in sorted_years:
                            if year in temporal_results[model_key]:
                                x_data.append(year)
                                y_data.append(temporal_results[model_key][year][metric_key] * 100)
                        
                        if len(x_data) > 0:
                            ax.plot(x_data, y_data, color=model_config['color'], 
                                   marker='o', linewidth=2, markersize=6,
                                   label=model_config['name'])
                
                ax.set_xlabel('Year', fontsize=12)
                ax.set_ylabel(ylabel, fontsize=12)
                ax.set_title(f'{ylabel.replace(" (%)", "")} - Temporal Comparison', 
                           fontsize=14, fontweight='bold')
                ax.legend(loc='best')
                ax.grid(True, alpha=0.3)
                
                # Evidenzia periodo COVID se presente
                covid_years = [2020, 2021]
                if any(year in sorted_years for year in covid_years):
                    for covid_year in covid_years:
                        if covid_year in sorted_years:
                            ax.axvline(covid_year, color='red', linestyle='--', 
                                     alpha=0.5, linewidth=1)
                    ax.text(0.02, 0.98, 'Red lines: COVID-19 period', 
                           transform=ax.transAxes, fontsize=10, 
                           verticalalignment='top', 
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
                
                plt.tight_layout()
                
                # Salva
                output_path = Path(OUTPUT_DIRS['comparison']) / filename
                plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
                print(f"💾 Salvato: {output_path}")
                
                plt.show()
            
            # Analisi di stabilità temporale
            print("\n📊 ANALISI STABILITÀ TEMPORALE:")
            print("-" * 50)
            
            for model_key in successfully_loaded:
                if model_key in temporal_results and len(temporal_results[model_key]) > 1:
                    model_name = MODELS[model_key]['name']
                    
                    # Calcola variabilità per MRR
                    mrr_values = [temporal_results[model_key][year]['mrr'] * 100 
                                 for year in temporal_results[model_key].keys()]
                    mrr_std = np.std(mrr_values)
                    mrr_mean = np.mean(mrr_values)
                    stability_score = mrr_mean / (mrr_std + 1e-6)  # Higher = more stable
                    
                    print(f"{model_name:>15s}: MRR avg={mrr_mean:.1f}%, std={mrr_std:.1f}%, stability={stability_score:.1f}")
            
            print("\n✅ Analisi temporale completata!")

## 🔍 5. Analisi Errori Comparativa

Confronto dei pattern di errore tra i diversi modelli per identificare bias e differenze qualitative.

In [ ]:
# ============================================================================
# ANALISI ERRORI COMPARATIVA
# ============================================================================

if len(successfully_loaded) < 2:
    print("⏭️  Saltando analisi errori: servono almeno 2 modelli")
else:
    print("🔍 Analisi comparativa degli errori...")
    
    # Analizza i POI più problematici per ogni modello
    print("\n📊 POI PIÙ PROBLEMATICI PER MODELLO:")
    print("=" * 60)
    
    model_error_patterns = {}
    
    for model_key in successfully_loaded:
        df = model_data[model_key]
        model_name = MODELS[model_key]['name']
        
        if 'ground_truth_norm' in df.columns and 'prediction_norm' in df.columns:
            # Calcola hit@1 se non presente
            if 'hit@1' not in df.columns:
                df['hit@1'] = df['prediction_norm'].str[0] == df['ground_truth_norm']
            
            # Analizza errori
            errors_df = df[~df['hit@1']].copy()
            
            if len(errors_df) > 0:
                # Top POI problematici (ground truth con più errori)
                error_counts = errors_df['ground_truth_norm'].value_counts().head(5)
                total_counts = df['ground_truth_norm'].value_counts()
                
                print(f"\n🔴 {model_name}:")
                print("   Top POI con più errori:")
                
                error_data = []
                for poi, error_count in error_counts.items():
                    total_count = total_counts.get(poi, error_count)
                    error_rate = error_count / total_count
                    print(f"     {poi[:25]:>25s}: {error_count:>4d} errori ({error_rate:>5.1%})")
                    error_data.append((poi, error_count, error_rate))
                
                model_error_patterns[model_key] = error_data
    
    # Confronto bias geografici
    print(f"\n📊 CONFRONTO BIAS NELLE PREDIZIONI:")
    print("=" * 60)
    
    for model_key in successfully_loaded:
        df = model_data[model_key]
        model_name = MODELS[model_key]['name']
        
        if 'prediction_norm' in df.columns:
            # Analizza predizioni più frequenti (possibili bias)
            all_predictions = []
            for pred_list in df['prediction_norm']:
                if isinstance(pred_list, list) and len(pred_list) > 0:
                    all_predictions.append(pred_list[0])  # Solo top-1 prediction
            
            if all_predictions:
                pred_freq = Counter(all_predictions)
                top_predictions = pred_freq.most_common(5)
                
                print(f"\n🎯 {model_name} - Predizioni più frequenti:")
                total_preds = len(all_predictions)
                for poi, count in top_predictions:
                    percentage = (count / total_preds) * 100
                    print(f"     {poi[:25]:>25s}: {count:>5d} ({percentage:>4.1f}%)")
    
    # Matrice di confusione comparativa (se fattibile)
    if len(successfully_loaded) >= 2:
        print(f"\n📊 Generazione matrici di confusione per confronto...")
        
        # Identifica POI comuni più frequenti
        all_gt_pois = set()
        for model_key in successfully_loaded:
            df = model_data[model_key]
            if 'ground_truth_norm' in df.columns:
                all_gt_pois.update(df['ground_truth_norm'].unique())
        
        # Prendi top 10 POI più comuni
        poi_frequencies = Counter()
        for model_key in successfully_loaded:
            df = model_data[model_key]
            if 'ground_truth_norm' in df.columns:
                poi_frequencies.update(df['ground_truth_norm'].value_counts().to_dict())
        
        top_pois = [poi for poi, _ in poi_frequencies.most_common(10)]
        
        # Crea confusion matrix per ogni modello
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, model_key in enumerate(successfully_loaded):
            if i >= len(axes):
                break
                
            df = model_data[model_key]
            model_name = MODELS[model_key]['name']
            ax = axes[i]
            
            if 'ground_truth_norm' in df.columns and 'prediction_norm' in df.columns:
                # Filtra per top POI
                mask = (df['ground_truth_norm'].isin(top_pois) & 
                       df['prediction_norm'].str[0].isin(top_pois))
                df_filtered = df[mask]
                
                if len(df_filtered) > 0:
                    # Crea confusion matrix
                    cm = pd.crosstab(df_filtered['ground_truth_norm'],
                                   df_filtered['prediction_norm'].str[0],
                                   normalize='index')
                    
                    # Plot heatmap
                    sns.heatmap(cm, annot=True, fmt='.2f', cmap='Blues', 
                              ax=ax, cbar_kws={'shrink': 0.5})
                    ax.set_title(f'{model_name}\nConfusion Matrix', fontweight='bold')
                    ax.set_xlabel('Predicted')
                    ax.set_ylabel('True')
                    
                    # Ruota etichette per leggibilità
                    ax.tick_params(axis='x', rotation=45)
                    ax.tick_params(axis='y', rotation=0)
            else:
                ax.text(0.5, 0.5, f'{model_name}\nDati non disponibili', 
                       ha='center', va='center', transform=ax.transAxes)
        
        # Nascondi assi non utilizzati
        for j in range(len(successfully_loaded), len(axes)):
            axes[j].set_visible(False)
        
        plt.suptitle('Confusion Matrices Comparison - Top 10 POI', 
                    fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        # Salva
        output_path = Path(OUTPUT_DIRS['comparison']) / 'confusion_matrices_comparison.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"💾 Salvato: {output_path}")
        
        plt.show()
    
    print("\n✅ Analisi errori comparativa completata!")

## 📋 6. Report Finale e Riepilogo

Genera un report completo con tutti i risultati e le raccomandazioni per l'integrazione nella tesi.

In [ ]:
# ============================================================================
# REPORT FINALE E RIEPILOGO
# ============================================================================

print("📋 Generazione report finale...\n")
print("=" * 80)
print("🎓 MULTI-MODEL COMPARISON ANALYSIS - FINAL REPORT")
print("=" * 80)

if len(successfully_loaded) == 0:
    print("\n❌ NESSUN MODELLO CARICATO - VERIFICA CONFIGURAZIONE")
    print("\n💡 AZIONI RICHIESTE:")
    print("   1. Crea la directory results/ nella root del progetto")
    print("   2. Crea sottodirectory per ogni modello (es. results/llama3.1_8b/)")
    print("   3. Sposta i file CSV in quelle directory")
    print("   4. Ri-esegui questo notebook")
else:
    # Riepilogo modelli caricati
    print(f"\n📊 MODELLI ANALIZZATI: {len(successfully_loaded)}")
    print("-" * 50)
    
    total_predictions = 0
    for model_key in successfully_loaded:
        model_config = MODELS[model_key]
        metrics = model_metrics[model_key]
        total_predictions += metrics['total_predictions']
        
        print(f"✅ {model_config['name']:>15s}: {metrics['total_predictions']:>6,} predizioni")
    
    print(f"\n📈 TOTALE PREDIZIONI ANALIZZATE: {total_predictions:,}")
    
    # Best performers
    print(f"\n🏆 BEST PERFORMERS PER METRICA:")
    print("-" * 50)
    
    metrics_names = [
        ('top1_accuracy', 'Top-1 Accuracy'),
        ('top5_hit_rate', 'Top-5 Hit Rate'),
        ('mrr', 'Mean Reciprocal Rank')
    ]
    
    for metric_key, metric_name in metrics_names:
        best_model = max(successfully_loaded, key=lambda k: model_metrics[k][metric_key])
        best_value = model_metrics[best_model][metric_key] * 100
        print(f"{metric_name:>18s}: {MODELS[best_model]['name']} ({best_value:.1f}%)")
    
    # Efficienza se disponibile
    models_with_time = [k for k in successfully_loaded 
                       if model_metrics[k]['processing_time_mean'] > 0]
    
    if models_with_time:
        fastest_model = min(models_with_time, 
                           key=lambda k: model_metrics[k]['processing_time_mean'])
        fastest_time = model_metrics[fastest_model]['processing_time_mean']
        print(f"{'Efficienza':>18s}: {MODELS[fastest_model]['name']} ({fastest_time:.1f}s/card)")
    
    # Files generati
    print(f"\n📁 FILES GENERATI:")
    print("-" * 50)
    
    comparison_dir = Path(OUTPUT_DIRS['comparison'])
    generated_files = list(comparison_dir.glob('*.png')) + list(comparison_dir.glob('*.csv')) + list(comparison_dir.glob('*.tex'))
    
    print(f"Directory: {comparison_dir}")
    for file_path in sorted(generated_files):
        file_size = file_path.stat().st_size / 1024  # KB
        print(f"  ✅ {file_path.name} ({file_size:.1f} KB)")
    
    # Istruzioni per LaTeX
    print(f"\n📄 INTEGRAZIONE LATEX:")
    print("-" * 50)
    print("1. I grafici sono salvati in img/multi_model_comparison/")
    print("2. La tabella LaTeX è in performance_table.tex")
    print("3. Sostituisci i placeholder --% nella tabella del .tex con i dati reali")
    print("4. Decommenta le linee \\includegraphics nel file .tex")
    
    # Raccomandazioni
    print(f"\n💡 RACCOMANDAZIONI:")
    print("-" * 50)
    
    if len(successfully_loaded) >= 3:
        # Identifica modelli complementari
        best_accuracy = max(successfully_loaded, key=lambda k: model_metrics[k]['top1_accuracy'])
        best_efficiency = min(models_with_time, key=lambda k: model_metrics[k]['processing_time_mean']) if models_with_time else None
        
        print(f"🎯 Per accuratezza massima: {MODELS[best_accuracy]['name']}")
        if best_efficiency:
            print(f"⚡ Per efficienza massima: {MODELS[best_efficiency]['name']}")
        
        # Analisi trade-off
        if models_with_time and len(models_with_time) >= 2:
            print("\n🔄 TRADE-OFF ANALYSIS:")
            for model_key in models_with_time:
                model_name = MODELS[model_key]['name']
                accuracy = model_metrics[model_key]['top1_accuracy'] * 100
                time_per_card = model_metrics[model_key]['processing_time_mean']
                efficiency_score = accuracy / time_per_card  # accuracy per second
                print(f"  {model_name:>15s}: {efficiency_score:.2f} accuracy/second")
    
    # Status completamento
    print(f"\n✅ ANALISI COMPLETATA CON SUCCESSO!")
    print(f"   📊 {len(successfully_loaded)} modelli analizzati")
    print(f"   📈 {len(generated_files)} file generati")
    print(f"   🎯 Pronti per integrazione nella tesi")

print("\n" + "=" * 80)
print("🎓 MULTI-MODEL ANALYSIS COMPLETED")
print("=" * 80)

## 📝 Note Finali e Prossimi Passi

### ✅ Checklist Completamento

- [ ] **Dati caricati**: Tutti i modelli hanno dati in `results/`
- [ ] **Grafici generati**: Files PNG creati in `img/multi_model_comparison/`
- [ ] **Tabella pronta**: File LaTeX generato per integrazione
- [ ] **Analisi completata**: Report finale visualizzato

### 🔄 Prossimi Passi

1. **Verifica grafici**: Controlla che tutti i PNG siano stati generati correttamente
2. **Integra LaTeX**: Copia il contenuto di `performance_table.tex` nel tuo file principale
3. **Decommenta immagini**: Rimuovi i `%` dalle linee `\includegraphics` nel .tex
4. **Compila tesi**: Verifica che LaTeX compili senza errori
5. **Revisiona risultati**: Controlla che i grafici supportino la narrativa della tesi

### ⚠️ Troubleshooting

- **Nessun modello caricato**: Verifica struttura directory `results/`
- **Grafici mancanti**: Controlla che directory `img/` esista e sia scrivibile
- **Errori LaTeX**: Verifica path delle immagini (`../../img/...`)
- **Performance inattese**: Controlla formato e qualità dei dati CSV

---

**🎉 Congratulazioni!** Hai ora un sistema completo per confrontare le performance dei tuoi modelli LLM sulla predizione di mobilità turistica. I grafici e le analisi generate sono pronti per l'inclusione nella tua tesi di laurea.